In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import mdtraj as md
import numpy as np
from perses.app.relative_point_mutation_setup import PointMutationExecutor

from openmmtools.integrators import LangevinIntegrator
from simtk.openmm import unit, app

# Set up logger
import logging
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)

INFO:numexpr.utils:Note: detected 80 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 80 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions


In [2]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/113/"
resid = "501"
name = "TYR"
state = 1
t_max = 1200
radius = 0.5
phase = 'complex'
length = 1
move_length = 1
timestep = 4
radius = 0.5

In [3]:
# Load rhtf
i = os.path.basename(os.path.dirname(outdir))
path = os.path.join(outdir, f"{i}_{phase}_{state}.pickle")
_logger.info(f"path: {path}")
htf = pickle.load(open(path, "rb" ))

INFO:root:path: /data/chodera/zhangi/perses_benchmark/neq/14/113/113_complex_1.pickle


In [5]:
# Minimize positions
_logger.info("Minimizing positions")
integrator = LangevinIntegrator(timestep=4.0 * unit.femtoseconds)
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(htf.hybrid_system, integrator, platform)
# context = openmm.Context(hybrid_system_copy, integrator, platform)
# lambda_alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
# lambda_protocol = LambdaProtocol(functions = 'default')
# lambda_alchemical_state.set_alchemical_parameters(1, lambda_protocol)
# thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(htf.hybrid_system, temperature=298 * unit.kelvin), composable_states=[lambda_alchemical_state])


INFO:root:Minimizing positions


In [ ]:
# context = thermodynamic_state.create_context(integrator, platform=platform)


In [6]:
context.setPositions(htf.hybrid_positions)
openmm.LocalEnergyMinimizer.minimize(context)
positions = context.getState(getPositions=True).getPositions()
positions_minimized = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)


In [7]:
app.PDBFile.writeFile(htf._topology_proposal.old_topology, htf.old_positions(positions_minimized), open(os.path.join(outdir, f"positions_minimized_old.pdb"), "w"), keepIds=True)
app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(positions_minimized), open(os.path.join(outdir, f"positions_minimized_new.pdb"), "w"), keepIds=True)

In [9]:
print(context.getState(getEnergy=True).getPotentialEnergy())

-5810070.91214643 kJ/mol


In [ ]:
# Build REST factory
_logger.info("Generating REST factory")
_logger.info(f"radius:{radius} nm")
for res in htf.hybrid_topology.residues:
    if res.resSeq == int(resid) and res.chain.index == 0:
        mutated_res = res
query_indices = [atom.index for atom in mutated_res.atoms]
_logger.info(f"query indices {query_indices}")
traj = md.Trajectory(np.array(positions_minimized), htf.hybrid_topology)
solute_atoms = list(traj.topology.select("is_protein"))
rest_atoms = list(md.compute_neighbors(traj, radius, query_indices, haystack_indices=solute_atoms)[0])
_logger.info(f"rest atoms {rest_atoms}")
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=rest_atoms, use_dispersion_correction=True)

In [10]:
with open(os.path.join(outdir, "113_complex_rest_factory.pickle"), "wb") as f:
    pickle.dump(factory, f)
with open(os.path.join(outdir, "minimized_positions.npy"), "wb") as f:
    np.save(f, positions_minimized)

In [4]:
with open(os.path.join(outdir, "113_complex_rest_factory.pickle"), "rb") as f:
    factory = pickle.load(f)
with open(os.path.join(outdir, "minimized_positions.npy"), "rb") as f:
    positions_minimized = np.load(f)

In [5]:
_logger.info("Generating REST states")
# Get REST system
REST_system = factory.REST_system

# Create states for each replica
n_replicas = 24  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = t_max * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

context_cache = cache.ContextCache()

INFO:root:Generating REST states


In [6]:
import time

In [7]:
# Create thermodynamics states
sampler_state =  SamplerState(positions_minimized, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
sampler_state_list = []
for temperature in temperatures:
    print(f"starting temp: {temperature}")
    initial_time = time.time()
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

    # now generating a sampler_state for each thermodynamic state, with relaxed positions
    # context, context_integrator = context_cache.get_context(compound_thermodynamic_state_copy)
    feptasks.minimize(compound_thermodynamic_state_copy, sampler_state, max_iterations=0)
    sampler_state_list.append(copy.deepcopy(sampler_state))
    elapsed_time = (time.time() - initial_time) * unit.seconds
    print(f"elapsed time: {elapsed_time}")

starting temp: 298.0 K
elapsed time: 212.58193469047546 s
starting temp: 321.3270432882694 K
elapsed time: 189.46807765960693 s
starting temp: 345.6906771384804 K
elapsed time: 190.83197569847107 s
starting temp: 371.1369648249121 K
elapsed time: 281.1197576522827 s
starting temp: 397.71401654887063 K
elapsed time: 188.18447375297546 s
starting temp: 425.4720803985772 K
elapsed time: 188.74381375312805 s
starting temp: 454.4636373510657 K
elapsed time: 280.0181608200073 s
starting temp: 484.7435004957092 K
elapsed time: 187.4496133327484 s
starting temp: 516.3689186669689 K
elapsed time: 189.01001453399658 s
starting temp: 549.3996846823022 K
elapsed time: 186.83598041534424 s
starting temp: 583.898248389871 K
elapsed time: 279.67587995529175 s
starting temp: 619.929834739782 K
elapsed time: 187.77266144752502 s
starting temp: 657.5625671020932 K
elapsed time: 187.35524463653564 s
starting temp: 696.8675960647399 K
elapsed time: 279.33665323257446 s
starting temp: 737.9192339548888 K
e

In [9]:
with open(os.path.join(outdir, "sampler_state_list.pickle"), "wb") as f:
    pickle.dump(sampler_state_list, f)

In [5]:
with open(os.path.join(outdir, "sampler_state_list.pickle"), "rb") as f:
    sampler_state_list = pickle.load(f)

In [6]:
for i, ss in enumerate(sampler_state_list):
    app.PDBFile.writeFile(htf._topology_proposal.old_topology, htf.old_positions(ss.positions), open(os.path.join(outdir, f"{i}_old.pdb"), "w"), keepIds=True)
    app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(ss.positions), open(os.path.join(outdir, f"{i}_new.pdb"), "w"), keepIds=True)

In [15]:
htf.hybrid_positions

Quantity(value=array([[-3.3747    ,  1.7107    ,  4.8754    ],
       [-3.2953    ,  1.6332    ,  4.9304    ],
       [-3.363     ,  1.8605    ,  4.8939    ],
       ...,
       [-2.9511449 ,  2.00742073,  0.43268726],
       [-3.11468275,  1.82748376,  0.37035705],
       [-3.03894171,  1.83670447,  0.44679181]]), unit=nanometer)

In [20]:
app.PDBFile.writeFile(htf._topology_proposal.old_topology, htf.old_positions(unit.Quantity(value=positions_minimized, unit=unit.nanometers)), open(os.path.join(outdir, f"positions_minimized_old.pdb"), "w"), keepIds=True)
app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(unit.Quantity(value=positions_minimized, unit=unit.nanometers)), open(os.path.join(outdir, f"positions_minimized_new.pdb"), "w"), keepIds=True)

In [ ]:
from openmmtools.multistate import ReplicaExchangeSampler
import mpiplus
class ReplicaExchangeSampler2(ReplicaExchangeSampler):
    @mpiplus.on_single_node(rank=0, broadcast_result=False, sync_nodes=False)
    @mpiplus.delayed_termination
    def _report_iteration_items(self):
        """
        Sub-function of :func:`_report_iteration` which handles all the actual individual item reporting in a
        sub-class friendly way. The final actions of writing timestamp, last-good-iteration, and syncing
        should be left to the :func:`_report_iteration` and subclasses should extend this function instead
        """
        replica_id = np.where(self._replica_thermodynamic_states == 0)[0][0]
        print("ITERATION: ", self._iteration)
        print("REPLICA THERMOSTATES ", self._replica_thermodynamic_states, type(self._replica_thermodynamic_states))
        print("REPLICA ID ", replica_id, type(replica_id))
        self._reporter.write_sampler_states([self._sampler_states[replica_id]], self._iteration)

        self._reporter.write_replica_thermodynamic_states(self._replica_thermodynamic_states, self._iteration)
        self._reporter.write_mcmc_moves(self._mcmc_moves)  # MCMCMoves can store internal statistics.
        self._reporter.write_energies(self._energy_thermodynamic_states, self._neighborhoods, self._energy_unsampled_states,
                                      self._iteration)
        self._reporter.write_mixing_statistics(self._n_accepted_matrix, self._n_proposed_matrix, self._iteration)


In [ ]:
# Set up sampler
_logger.setLevel(logging.DEBUG)
_logger.info("About to start repex")
print(f"move steps: {int((move_length*1000)/timestep)}")
print(f"timestep: {timestep} fs")
move = mcmc.LangevinSplittingDynamicsMove(timestep=timestep*unit.femtoseconds, n_steps=int((move_length*1000)/timestep))
simulation = ReplicaExchangeSampler2(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(outdir, f"{i}_{phase}_{name.lower()}_{length}ns.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state_list,
                  storage=reporter)
simulation.run()

# Generate htf for ALA->TYR

In [53]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys

In [54]:
atp, system_generator = generate_atp()

In [61]:
htf = generate_dipeptide_top_pos_sys(atp.topology, 'TYR', atp.system, atp.positions, system_generator, validate_energy_bookkeeping=False, conduct_htf_prop = True, flatten_exceptions=True, flatten_torsions=True, validate_endstate_energy=False)

INFO:proposal_generator:	Conducting polymer point mutation proposal...


making topology proposal


INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD1, atomic number=6), Atom(name=CD2, atomic number=6), Atom(name=CE1, atomic number=6), Atom(name=CE2, atomic number=6), Atom(name=CZ, atomic number=6), Atom(name=OH, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD1, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HE1, atomic number=1), Atom(name=HE2, atomic number=1), Atom(name=HH, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 15
INFO:geometry:Atom index proposal order is [10, 1

generating geometry engine
making geometry proposal from ALA to TYR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

INFO:geometry:total reduced potential before atom placement: 9.116481141712038


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.67298667212561
INFO:geometry:final reduced energy 29.78946596595153
INFO:geometry:sum of energies: 29.789467813837646
INFO:geometry:magnitude of difference in the energies: 1.8478861179005435e-06
INFO:geometry:Final logp_proposal: -26942.869531381337


added energy components: [('CustomBondForce', 0.0005202039273265048), ('CustomAngleForce', 0.45111977562481365), ('CustomTorsionForce', 7.25046332853325), ('CustomBondForce', 12.970883364040219)]


INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanome

In [62]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, htf.new_positions(htf.hybrid_positions), open("tyr.pdb", "w"))

# Generate htf allowing for multiple geometry proposals

In [11]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
import os
import pickle
from openmmtools.integrators import LangevinIntegrator 



In [2]:
from __future__ import absolute_import

from perses.utils.openeye import createOEMolFromSDF, extractPositionsFromOEMol
from perses.annihilation.relative import HybridTopologyFactory, RepartitionedHybridTopologyFactory
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.rjmc.geometry import FFAllAngleGeometryEngine

import simtk.openmm as openmm
import simtk.openmm.app as app
import simtk.unit as unit
import numpy as np
from openmoltools import forcefield_generators
import mdtraj as md
from openmmtools.constants import kB
from perses.tests.utils import validate_endstate_energies
from openff.toolkit.topology import Molecule
from openmmforcefields.generators import SystemGenerator

ENERGY_THRESHOLD = 1e-2
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
ring_amino_acids = ['TYR', 'PHE', 'TRP', 'PRO', 'HIS']

# Set up logger
import logging
_logger = logging.getLogger("setup")
_logger.setLevel(logging.INFO)



In [5]:
outdir = "/data/chodera/zhangi/perses_benchmark/neq/14/113/"
with open("/data/chodera/zhangi/perses_benchmark/neq/14/113/113_complex.pickle", "rb") as f:
    htf = pickle.load(f)
    topology_proposal = htf._topology_proposal
    

In [12]:
positions = htf.old_positions(htf.hybrid_positions)

geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)

# Only validate energy bookkeeping if the WT and proposed residues do not involve rings
validate_bool = False

# Create geometry proposal
for i in range(5):
    new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=validate_bool)
    logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=validate_bool)
    # Check potential energy
    platform = openmm.Platform.getPlatformByName('Reference')
    integrator = LangevinIntegrator(temperature=temperature)
    context = openmm.Context(topology_proposal.new_system, integrator, platform)
    context.setPeriodicBoxVectors(*topology_proposal.new_system.getDefaultPeriodicBoxVectors())
    context.setPositions(new_positions)
    context.setVelocitiesToTemperature(temperature)
    potential_energy = context.getState(getEnergy=True).getPotentialEnergy()

    print("POTENTIAL ENERGY: ", potential_energy)
    app.PDBFile.writeFile(topology_proposal.new_topology, new_positions, open(os.path.join(outdir, f"minimized_{i}.pdb"), "w"), keepIds=True)
    

INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 15
INFO:geometry:Atom index proposal order is [2611, 2614, 2615, 2617, 2624, 2619, 2622, 2620, 2612, 2623, 2625, 2618, 2616, 2613, 2621]
INFO:geometry:omitted_bonds: [(2622, 2624)]
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 8439 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 29350 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 55146 torsions in reference force.
INFO:geometry:	creating nonbonded forc

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 15 new atoms
INFO:geometry:	reduced angle potential = 0.2490447547103794.
INFO:geometry:	reduced angle potential = 0.6304372383699829.
INFO:geometry:	reduced angle potential = 0.13437060169044188.
INFO:geometry:	reduced angle potential = 0.006128155223984333.
INFO:geometry:	reduced angle potential = 0.4036751228305759.
INFO:geometry:	reduced angle potential = 0.029156827278966332.
INFO:geometry:	reduced angle potential = 0.06516142210056118.
INFO:geometry:	reduced angle potential = 0.035813503118536885.
INFO:geometry:	reduced angle potential = 8.545518562010487e-05.
INFO:geometry:	reduced angle potential = 0.2943690178177199.
INFO:geometry:	reduced angle potential = 0.0015900631477493358.
INFO:geometry:	reduced angle potential = 1.1300828360503914.
INFO:geometry:	reduced angle potential = 0.0982284127309357.
INFO:geometry:	reduced angle potential = 0.10762167458574802.
INFO:geometry:	reduced angle potential = 0.005219459092383839.
INFO:geometry:	beginning constr

...

INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.006118641034753415.
INFO:geometry:	reduced angle potential = 0.5604275396387628.
INFO:geometry:	reduced angle potential = 0.07650810571585355.
INFO:geometry:	reduced angle potential = 0.167841485694284.
INFO:geometry:	reduced angle potential = 0.012217088956631855.
INFO:geometry:	reduced angle potential = 0.11607969104718184.
INFO:geometry:	reduced angle potential = 0.17311349100355702.
INFO:geometry:	reduced angle potential = 1.3306562413993697.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 8434 bond forces in the no-nonbonded final system
INFO:geome

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 23419.390629614598


conducting subsequent work with the following platform: CUDA


KeyboardInterrupt: 

In [14]:
pdb = app.PDBFile(os.path.join(outdir, "minimized_0.pdb"))




In [15]:
integrator = LangevinIntegrator(timestep=4.0 * unit.femtoseconds)
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')



In [16]:
context = openmm.Context(htf._topology_proposal.new_system, integrator, platform)
context.setPositions(pdb.positions)
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  409488352.9070268 kJ/mol


In [17]:
openmm.LocalEnergyMinimizer.minimize(context)

In [18]:
pos = context.getState(getPositions=True).getPositions()

In [20]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, pos, open("/data/chodera/zhangi/perses_benchmark/neq/14/113/complex_new_minimized_0.pdb", "w"), keepIds=True)

In [21]:
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  -5822689.320420526 kJ/mol


In [22]:
pdb = app.PDBFile(os.path.join(outdir, "minimized_3.pdb"))




In [23]:
integrator = LangevinIntegrator(timestep=4.0 * unit.femtoseconds)
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')



In [24]:
context = openmm.Context(htf._topology_proposal.new_system, integrator, platform)
context.setPositions(pdb.positions)
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  2315388684830571.5 kJ/mol


In [25]:
openmm.LocalEnergyMinimizer.minimize(context)

In [26]:
pos = context.getState(getPositions=True).getPositions()

In [27]:
app.PDBFile.writeFile(htf._topology_proposal.new_topology, pos, open("/data/chodera/zhangi/perses_benchmark/neq/14/113/complex_new_minimized_3.pdb", "w"), keepIds=True)

In [28]:
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  -5841484.500344999 kJ/mol


# Generate geometry proposals for ASN->TYR

In [21]:
from openmmforcefields.generators import SystemGenerator
from simtk import openmm
from simtk.openmm import app, unit
from openmmtools.integrators import LangevinIntegrator 
from perses.rjmc.geometry import FFAllAngleGeometryEngine
from perses.rjmc.topology_proposal import PointMutationEngine

from openmmtools.constants import kB
temperature = 300*unit.kelvin
# Compute kT and inverse temperature.
kT = kB * temperature
beta = 1.0 / kT

In [6]:
pdb = app.PDBFile("../../input/asn_vacuum.pdb")

In [4]:
forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']

In [7]:
barostat = None
system_generator = SystemGenerator(forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                small_molecule_forcefield='gaff-2.11',
                                molecules=None,
                                cache=None)

system = system_generator.create_system(pdb.topology) # Update the parametrization scheme to amberff14sb


In [9]:
integrator = LangevinIntegrator(timestep=4.0 * unit.femtoseconds)
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')



In [10]:
context = openmm.Context(system, integrator, platform)
context.setPositions(pdb.positions)
openmm.LocalEnergyMinimizer.minimize(context)

In [13]:
pos = context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(pdb.topology, pos, open("asn_minimized.pdb", "w"), keepIds=True)

In [15]:
# Create the point mutation engine
new_res = 'TYR'
point_mutation_engine = PointMutationEngine(wildtype_topology=pdb.topology,
                                            system_generator=system_generator,
                                            chain_id='A', # Denote the chain id allowed to mutate (it's always a string variable)
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'], # The residue ids allowed to mutate
                                            allowed_mutations=[('2', new_res)], # The residue ids allowed to mutate with the three-letter code allowed to change
                                            aggregate=True) # Always allow aggregation


In [16]:
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=pdb.topology)

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=ND2, atomic number=7), Atom(name=OD1, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD21, atomic number=1), Atom(name=HD22, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG, atomic number=6), Atom(name=CD1, atomic number=6), Atom(name=CD2, atomic number=6), Atom(name=CE1, atomic number=6), Atom(name=CE2, atomic number=6), Atom(name=CZ, atomic number=6), Atom(name=OH, atomic number=8), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1), Atom(name=HD1, atomic number=1), Atom(name=HD2, atomic number=1), Atom(name=HE1, atomic number=1), Atom(name=HE2, atomic number=1), Atom(name=HH, atomic number=1)]


In [19]:
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                       use_sterics=False,
                                       n_bond_divisions=100,
                                       n_angle_divisions=180,
                                       n_torsion_divisions=360,
                                       verbose=True,
                                       storage=None,
                                       bond_softening_constant=1.0,
                                       angle_softening_constant=1.0,
                                       neglect_angles = False,
                                       use_14_nonbondeds = True)




In [25]:
for i in range(5):
    forward_new_positions, logp_proposal = geometry_engine.propose(topology_proposal, pdb.positions, beta, validate_energy_bookkeeping=False)
    app.PDBFile.writeFile(topology_proposal.new_topology, forward_new_positions, open(f"tyr_proposal_{i}.pdb", "w"), keepIds=True)

INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 15
INFO:geometry:Atom index proposal order is [10, 15, 16, 25, 18, 20, 23, 21, 11, 12, 22, 17, 19, 24, 26]
INFO:geometry:omitted_bonds: [(20, 23)]
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 17 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 55 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 86 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference no

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 15 new atoms
INFO:geometry:	reduced angle potential = 0.018297205083098084.
INFO:geometry:	reduced angle potential = 0.0017512408455965297.
INFO:geometry:	reduced angle potential = 0.19804990154351074.
INFO:geometry:	reduced angle potential = 0.5786945224126003.
INFO:geometry:	reduced angle potential = 1.3891168557141718.
INFO:geometry:	reduced angle potential = 2.7509704781852733.
INFO:geometry:	reduced angle potential = 0.0004169074618967446.
INFO:geometry:	reduced angle potential = 0.027057048114269382.
INFO:geometry:	reduced angle potential = 0.01817911400534157.
INFO:geometry:	reduced angle potential = 1.346068800150258.
INFO:geometry:	reduced angle potential = 0.14026096819804518.
INFO:geometry:	reduced angle potential = 0.14060751990384923.
INFO:geometry:	reduced angle potential = 0.014249066433631918.
INFO:geometry:	reduced angle potential = 0.0005737253802920767.
INFO:geometry:	reduced angle potential = 0.001659243744820202.
INFO:geometry:	beginning con

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 9.117938551333578


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -25.024134537430477
INFO:geometry:final reduced energy 39.151656055183494
INFO:geometry:sum of energies: -15.906195986096899
INFO:geometry:magnitude of difference in the energies: 55.05785204128039
INFO:geometry:Final logp_proposal: 124.2221854088079


added energy components: [('CustomBondForce', 5.219378629139631), ('CustomAngleForce', 12.862085243167074), ('CustomTorsionForce', 9.227495056562395), ('CustomBondForce', -52.33309346629958)]


In [26]:
topology_proposal.core_new_to_old_atom_map

{6: 6,
 8: 8,
 13: 13,
 14: 14,
 7: 7,
 9: 9,
 0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 27: 20,
 28: 21,
 29: 22,
 30: 23,
 31: 24,
 32: 25}

In [28]:
for atom in pdb.topology.atoms():
    print(atom.index, atom)

0 <Atom 0 (H1) of chain 0 residue 0 (ACE)>
1 <Atom 1 (CH3) of chain 0 residue 0 (ACE)>
2 <Atom 2 (H2) of chain 0 residue 0 (ACE)>
3 <Atom 3 (H3) of chain 0 residue 0 (ACE)>
4 <Atom 4 (C) of chain 0 residue 0 (ACE)>
5 <Atom 5 (O) of chain 0 residue 0 (ACE)>
6 <Atom 6 (N) of chain 0 residue 1 (ASN)>
7 <Atom 7 (H) of chain 0 residue 1 (ASN)>
8 <Atom 8 (CA) of chain 0 residue 1 (ASN)>
9 <Atom 9 (HA) of chain 0 residue 1 (ASN)>
10 <Atom 10 (CB) of chain 0 residue 1 (ASN)>
11 <Atom 11 (HB2) of chain 0 residue 1 (ASN)>
12 <Atom 12 (HB3) of chain 0 residue 1 (ASN)>
13 <Atom 13 (C) of chain 0 residue 1 (ASN)>
14 <Atom 14 (O) of chain 0 residue 1 (ASN)>
15 <Atom 15 (CG) of chain 0 residue 1 (ASN)>
16 <Atom 16 (OD1) of chain 0 residue 1 (ASN)>
17 <Atom 17 (ND2) of chain 0 residue 1 (ASN)>
18 <Atom 18 (HD21) of chain 0 residue 1 (ASN)>
19 <Atom 19 (HD22) of chain 0 residue 1 (ASN)>
20 <Atom 20 (N) of chain 0 residue 2 (NME)>
21 <Atom 21 (H) of chain 0 residue 2 (NME)>
22 <Atom 22 (C) of chain 0 re